In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
%%capture
!pip install transformers
!pip install datasets
!pip install evaluate

In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
import os

In [4]:
dataset = load_dataset("json", data_files={"train": "/content/treino.jsonl", "test": "/content/teste.jsonl"})

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 500
    })
    test: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 100
    })
})

In [6]:
# Construção de embeddings do texto
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

mapDict = {
    "suporte": 0,
    "venda": 1
}

def transform_labels(example):
    completions = example['completion']

    result = []

    for item in completions:
        numero_label = mapDict[item]

        result.append(numero_label)

    return {"label": result}

def tokenize_function(example):
    return tokenizer(example['prompt'], padding=True, truncation=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [7]:
# Tokenização do input e transformação dos labels
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.map(transform_labels, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [8]:
from transformers import TrainingArguments

output_dir = "./bert-customer-service-test"

training_args = TrainingArguments(
    output_dir = output_dir,
    num_train_epochs = 3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    logging_dir = "./logs",
    logging_steps = 100,
    eval_strategy = "steps",
    eval_steps = 200,
    save_total_limit = 2,
    save_steps = 200,
    load_best_model_at_end = True,
    metric_for_best_model = "accuracy",
    report_to="none"
)

In [9]:
from transformers import Trainer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

os.environ['WANDB_DISABLE'] = "true"
os.environ['WANDB_MODE'] = "offline"

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metric(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis= -1)
    return metric.compute(predictions=predictions, references=labels)

In [11]:
trainer = Trainer(
    model,
    training_args,
    train_dataset = tokenized_datasets['train'],
    eval_dataset = tokenized_datasets['test'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metric
)

/tmp/ipython-input-2362678228.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [12]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss,Validation Loss


TrainOutput(global_step=189, training_loss=0.06196672311693272, metrics={'train_runtime': 551.4157, 'train_samples_per_second': 2.72, 'train_steps_per_second': 0.343, 'total_flos': 20041842366000.0, 'train_loss': 0.06196672311693272, 'epoch': 3.0})

In [13]:
trainer.evaluate()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.0005561147700063884,
 'eval_accuracy': 1.0,
 'eval_runtime': 20.4882,
 'eval_samples_per_second': 4.881,
 'eval_steps_per_second': 0.635,
 'epoch': 3.0}

In [14]:
trainer.save_model()

In [15]:
trainer.push_to_hub("luizaugustoliveira/modelservice")

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...ce-test/model.safetensors:   0%|          | 14.2kB /  438MB            

  ...ce-test/training_args.bin:  33%|###2      | 1.90kB / 5.78kB            

CommitInfo(commit_url='https://huggingface.co/luizaugustoliveira/bert-customer-service-test/commit/7cc41824bbb5b5027fab0311268e243688da4f94', commit_message='luizaugustoliveira/modelservice', commit_description='', oid='7cc41824bbb5b5027fab0311268e243688da4f94', pr_url=None, repo_url=RepoUrl('https://huggingface.co/luizaugustoliveira/bert-customer-service-test', endpoint='https://huggingface.co', repo_type='model', repo_id='luizaugustoliveira/bert-customer-service-test'), pr_revision=None, pr_num=None)

In [16]:
from transformers import pipeline

pipe = pipeline("text-classification", model="luizaugustoliveira/bert-customer-service-test")

config.json:   0%|          | 0.00/835 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cpu


In [18]:
pipe("Quero uma SmarTV 55 polegadas")

[{'label': 'LABEL_1', 'score': 0.9985326528549194}]

In [19]:
pipe("Não estou conseguindo fazer login no sistema")

[{'label': 'LABEL_0', 'score': 0.9993997812271118}]